In [5]:
import pandas as pd
import numpy as np
import plotly as py
from plotly import tools
import plotly.graph_objs as go
from feature_functions import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
from sklearn.linear_model import LinearRegression

# Variables:
future = 15
averages = [5,10,15,20,30,50,70,100,200,300]


# Loading the data
df = pd.read_csv("data/EURUSDhour.csv")
df.columns = ['date','open','high','low','close','volume']
df.date = pd.to_datetime(df.date, format='%d.%m.%Y %H:%M:%S.%f')
df = df.set_index(df.date)
df = df[['open','high','low','close','volume']]

# Drop elements where there is no mouvement (Market closed) like in the weekends
df = df.drop_duplicates(keep=False)
df

,open,high,low,close,volume
date,,,,,
2017-01-01 22:00:00,1.05236,1.05253,1.05221,1.05227,143.5400
2017-01-01 23:00:00,1.05227,1.05426,1.05226,1.05282,253.1400
2017-01-02 00:00:00,1.05283,1.05283,1.05268,1.05281,131.4000
2017-01-02 01:00:00,1.05282,1.05286,1.05225,1.05240,273.9400
2017-01-02 02:00:00,1.05239,1.05240,1.05164,1.05220,258.1400
2017-01-02 03:00:00,1.05219,1.05244,1.05184,1.05226,2288.1799
2017-01-02 04:00:00,1.05226,1.05232,1.05181,1.05193,711.8000
2017-01-02 05:00:00,1.05195,1.05197,1.05136,1.05161,288.2100
2017-01-02 06:00:00,1.05163,1.05176,1.05113,1.05172,390.8800


In [15]:
# Williams oscillator function

def williams(prices, periods):
    """

    :param prices: DataFrame of OHLC data
    :param periods: List of periods to calculate function value
    
    :return: williams oscillator function values
    """
    results = pd.DataFrame(index=prices.index)

    for i in range(len(periods)):
        Rs = []

        for j in range(periods[i], len(prices) - periods[i]):
            C = prices.close.iloc[j + 1]
            H = prices.high.iloc[j - periods[i]:j].max()
            L = prices.low.iloc[j - periods[i]:j].min()

            if H == L:
                R = 0
            else:
                R = - 100 * (H - C) / (H - L)
            Rs = np.append(Rs, R)

        df = pd.DataFrame(Rs, index=prices.iloc[periods[i] + 1:-periods[i] + 1].index)
        df.columns = ['williams ' + str(periods[i]) + ' R']
        df = df.dropna()

        results = pd.concat([results, df], axis=1)

    return results


In [16]:
williamsKey = [6, 7, 8, 9, 10]

williams(df, williamsKey)


,williams 6 R,williams 7 R,williams 8 R,williams 9 R,williams 10 R
date,,,,,
2017-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-01 23:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 01:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 02:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 03:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN
2017-01-02 05:00:00,-101.145038,NaN,NaN,NaN,NaN
2017-01-02 06:00:00,-96.946565,-96.946565,NaN,NaN,NaN


In [ ]:
# The data
interval = 20
run_length = 100
correct_predictions = 0
for i in range(run_length):
    X = df_with_averages.iloc[:df_with_averages.shape[0]- (i*interval),:-1]
    y = df_with_averages.result

    # Split the data
    X_train= X.iloc[:X.shape[0]-1,:]
    X_test= X.iloc[X.shape[0]-1:X.shape[0],:]
    y_train= y[:X.shape[0]-1]
    y_test =y [X.shape[0]-1:X.shape[0]]

    clf = GradientBoostingClassifier(random_state=5, learning_rate=0.01, n_estimators=10000)
    clf.fit(X_train, y_train) 

    # Predictions
    predicted = clf.predict(X_test)
    print (str(accuracy_score(y_test, predicted)))
    correct_predictions += accuracy_score(y_test, predicted)